# OpenAI and Local LLM Exploration
Last updated: March 13, 2024

## `llm` Python package

First we'll use Simon Willison's open-source `llm` Python package to interface with various LLM models, following the [My LLM CLI tool now supports self-hosted language models via plugins](https://simonwillison.net/2023/Jul/12/llm/) post and a [GPT4All model](https://gpt4all.io/index.html):

```shell
# prerequisite: git-lfs for cloning Git repos with large files/models
$ brew install git-lfs
# install llm package and gpt4all plugin
$ pip install llm
$ llm install llm-gpt4all
```

### CLI

In [1]:
! llm models

# See available models

OpenAI Chat: gpt-3.5-turbo (aliases: 3.5, chatgpt)
OpenAI Chat: gpt-3.5-turbo-16k (aliases: chatgpt-16k, 3.5-16k)
OpenAI Chat: gpt-4 (aliases: 4, gpt4)
OpenAI Chat: gpt-4-32k (aliases: 4-32k)
OpenAI Chat: gpt-4-1106-preview
OpenAI Chat: gpt-4-0125-preview
OpenAI Chat: gpt-4-turbo-preview (aliases: gpt-4-turbo, 4-turbo, 4t)
OpenAI Completion: gpt-3.5-turbo-instruct (aliases: 3.5-instruct, chatgpt-instruct)
gpt4all: all-MiniLM-L6-v2-f16 - SBert, 43.76MB download, needs 1GB RAM (installed)
gpt4all: gpt4all-falcon-newbpe-q4_0 - GPT4All Falcon, 3.92GB download, needs 8GB RAM (installed)
gpt4all: replit-code-v1_5-3b-newbpe-q4_0 - Replit, 1.82GB download, needs 4GB RAM
gpt4all: orca-mini-3b-gguf2-q4_0 - Mini Orca (Small), 1.84GB download, needs 4GB RAM
gpt4all: orca-2-7b - Orca 2 (Medium), 3.56GB download, needs 8GB RAM
gpt4all: rift-coder-v0-7b-q4_0 - Rift coder, 3.56GB download, needs 8GB RAM
gpt4all: mpt-7b-chat-newbpe-q4_0 - MPT Chat, 3.64GB download, needs 8GB RAM
gpt4all: em_german_mist

In [2]:
%%time

! llm -m gpt4all-falcon-newbpe-q4_0 "The capital of North Carolina?"

# Test falcon-newbpe-q4_0 model with CLI tool

The capital of North Carolina is Raleigh.
CPU times: user 48.4 ms, sys: 29 ms, total: 77.4 ms
Wall time: 14.4 s


In [3]:
%%time

! llm -m gpt4all-falcon-newbpe-q4_0 "The capital of North Carolina? Answer in one word"

# Sometimes this gives Charlotte? 😮

Charlotte
CPU times: user 12.6 ms, sys: 8.39 ms, total: 21 ms
Wall time: 2.81 s


### Python

We can use the [Python API](https://llm.datasette.io/en/stable/python-api.html) too.

In [4]:
import os

import llm
import requests
from strip_tags import strip_tags

In [5]:
%%time

model = llm.get_model("gpt4all-falcon-newbpe-q4_0")
response = model.prompt("The capital of North Carolina?")
response.text()

CPU times: user 185 ms, sys: 434 ms, total: 618 ms
Wall time: 1.3 s


'The capital of North Carolina is Raleigh.'

## ChatGPT 3.5 Turbo

In [6]:
gpt35_model = llm.get_model("gpt-3.5-turbo")
gpt35_model.key = os.getenv("OPENAI_API_KEY")

In [7]:
%%time

response = gpt35_model.prompt("The capital of North Carolina?")
response.text()

CPU times: user 43.4 ms, sys: 16.3 ms, total: 59.7 ms
Wall time: 778 ms


'The capital of North Carolina is Raleigh.'

## Traffic law enforcement statistics statute

Warning: Be careful with OpenAI costs! https://platform.openai.com/usage

In [6]:
resp = requests.get("https://www.ncleg.gov/EnactedLegislation/Statutes/HTML/BySection/Chapter_143B/GS_143B-903.html")
gs_143b_903 = strip_tags(resp.content)

In [7]:
%%time

response = gpt35_model.prompt(gs_143b_903, system="summerize")
response.text()

CPU times: user 212 ms, sys: 24.7 ms, total: 237 ms
Wall time: 4.38 s


'This section of the law outlines the requirements for the collection of traffic law enforcement statistics in North Carolina. The Department of Public Safety is responsible for collecting and maintaining information regarding traffic law enforcement by law enforcement officers, including the number of drivers stopped, identifying characteristics of drivers, alleged traffic violations, search details, citations issued, arrests made, use of force, injuries, and more. The information must be submitted to the Department within 60 days of the close of each month, and agencies that fail to comply may be ineligible for certain law enforcement grants. Law enforcement officers are assigned anonymous identification numbers, with the correlation between identification numbers and officer names being kept confidential. The Department must publish and distribute a list of officers subject to these provisions each year by December 1.'

In [13]:
%%time

response = model.prompt(gs_143b_903, system="summerize")
response.text()

CPU times: user 405 ms, sys: 526 ms, total: 932 ms
Wall time: 17.2 s


'The information required by this section is collected by law enforcement officers in the course of their duties to assist in traffic law enforcement and crime prevention. The data collected includes the number of drivers stopped for routine traffic enforcement, identifying characteristics of the drivers stopped, the alleged traffic violation that led to the stop, whether a search was instituted as a result of the stop, whether any contraband was found and the type and amount of any such contraband, whether an arrest was made as a result of either the stop or the search, whether any property was seized, with a description of that property, whether the officers making the stop encountered any physical resistance from the driver or passenger or passengers, whether the officers making the stop engaged in the use of force against the driver, passenger, or passengers, whether any injuries resulted from the stop, whether the circumstances surrounding the stop were the subject of any investig

* https://www.ncleg.gov/EnactedLegislation/Statutes/HTML/BySection/Chapter_143B/GS_143B-903.html
* https://github.com/vince-lam/awesome-local-llms